In [1]:
from typing import List, Any
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Run the previous notebooks to load environment variables, imports, embeddings, chunked texts, vector stores, RAG chains, and conversational RAG chains
%run "conversational_ragchain.ipynb"

def get_aggregated_answer_and_sources(input_question, session_id="abc123"):
    def is_invalid_answer(answer):
        return "I don't" in answer or "I'm sorry" in answer

    def get_response_and_sources(response, all_texts):
        answer = response["answer"]
        context = response["context"]
        used_chunks = [doc for doc in all_texts if any(chunk.page_content in doc.page_content for chunk in context)]
        sources = {(chunk.metadata['file'], chunk.metadata.get('page', 'Unknown')) for chunk in used_chunks}
        return answer, context, sources

    all_responses = {}
    all_contexts = []
    all_sources = set()

    for key in ['chroma', 'faiss', 'pinecone']:
        response = conversational_rag_chains[key].invoke(
            {"input": input_question},
            config={"configurable": {"session_id": session_id}},
        )
        if not is_invalid_answer(response["answer"]):
            answer, context, sources = get_response_and_sources(response, all_texts[key])
            all_responses[key] = answer
            all_contexts.extend(context)
            all_sources.update(sources)

    if not all_responses:
        return "I don't know the answer.", {}, [], set()

    # Combine all the valid responses into one, filtering out duplicates
    sentences = []
    for answer in all_responses.values():
        sentences.extend(answer.split(". "))

    # Embed the sentences to filter out duplicates
    sentence_embeddings = embedding_function.embed_documents(sentences)
    unique_sentences = []
    unique_embeddings = []

    for i, embedding in enumerate(sentence_embeddings):
        if not unique_embeddings:  # First sentence
            unique_sentences.append(sentences[i])
            unique_embeddings.append(embedding)
        else:
            similarities = cosine_similarity([embedding], unique_embeddings)
            if all(sim < 0.8 for sim in similarities[0]):  # Adjust the threshold as needed
                unique_sentences.append(sentences[i])
                unique_embeddings.append(embedding)

    aggregated_answer = ". ".join(unique_sentences)

    return aggregated_answer, all_responses, all_contexts, all_sources

def format_context(context: List[Any]) -> str:
    formatted_context = "\n\n".join([
        f"File: {doc.metadata['file']}, Page {doc.metadata.get('page', 'Unknown')}: {doc.page_content}"
        for doc in context
    ])
    return formatted_context


C:\Users\vlad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\vlad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [2]:
# Usage Example
question = "What are the primary tasks performed by broadcast technicians during radio and television broadcasts?"
aggregated_answer, all_responses, context, sources = get_aggregated_answer_and_sources(question)

# Format the output in a readable way
formatted_context = "\n\n".join([f"File: {doc.metadata['file']}, Page {doc.metadata.get('page', 'Unknown')}: {doc.page_content}" for doc in context])

output = f"""
Final Aggregated Answer: {aggregated_answer}

Intermediate Answers:
Chroma: {all_responses.get('chroma', 'No valid answer found')}
FAISS: {all_responses.get('faiss', 'No valid answer found')}
Pinecone: {all_responses.get('pinecone', 'No valid answer found')}

Context used for answer:
{formatted_context}
"""

print(output)


Parent run 1af4fb87-25f6-4a1c-bbd3-ac411c7d7a01 not found for run b8a631b0-abe3-45d8-ad11-39768e744ce5. Treating as a root run.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Final Aggregated Answer: Broadcast technicians during radio and television broadcasts are responsible for maintaining programming logs as required by station management and the Federal Communications Commission, controlling audio equipment to regulate volume and sound quality, monitoring the strength, clarity, and reliability of incoming and outgoing signals and adjusting equipment as necessary, regulating the fidelity, brightness, and contrast of video transmissions, previewing scheduled programs to ensure readiness for transmission, and selecting sources for programming reception or transmission. They also report and address equipment problems, record sound onto tape or film, align antennas with receiving dishes, and substitute programs when signals fail.. The primary tasks performed by broadcast technicians during radio and television broadcasts include:

1. Maintaining programming logs as required by station management and the Federal Communications Commission.
2. Controlling audi

In [3]:
# Use the get_aggregated_answer_and_sources function with the specified question
question = "What are the specific authorities and responsibilities of the Inspector General within the Department of Justice?"
aggregated_answer, all_responses, context, sources = get_aggregated_answer_and_sources(question)

# Format the output in a readable way
formatted_context = "\n\n".join([f"File: {doc.metadata['file']}, Page {doc.metadata.get('page', 'Unknown')}: {doc.page_content}" for doc in context])

output = f"""
Final Aggregated Answer: {aggregated_answer}

Intermediate Answers:
Chroma: {all_responses.get('chroma', 'No valid answer found')}
FAISS: {all_responses.get('faiss', 'No valid answer found')}
Pinecone: {all_responses.get('pinecone', 'No valid answer found')}

Context used for answer:
{formatted_context}
"""

print(output)

Parent run 04e6b224-d47b-4849-b640-1f5a29da266d not found for run e2fcab26-49f3-4862-a060-0bea5f444b59. Treating as a root run.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Parent run 8d234903-63f5-4a61-a27e-39a8afb69db8 not found for run 548d2994-607b-4f37-bda0-639594942049. Treating as a root run.


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Final Aggregated Answer: The specific authorities and responsibilities of the Inspector General within the Department of Justice include:

1. Conducting investigations and issuing reports relating to criminal wrongdoing and administrative misconduct of Department employees and administration of programs and operations, as deemed necessary or desirable.
2. Receiving and investigating complaints or information from employees concerning possible violations of law, rules, regulations, mismanagement, gross waste of funds, abuse of authority, or substantial and specific dangers to public health and safety.
3. Having direct and prompt access to the Attorney General for any matters pertaining to the performance of the functions and responsibilities of the Office of the Inspector General (OIG).
4. Having access to all records, reports, audits, reviews, documents, papers, recommendations, or other material available to the Department and its components that relate to programs and operations wit

In [4]:
# Use the get_aggregated_answer_and_sources function with the specified question
question = "What were the forecasts for U.S. crude oil production in the fourth quarter of 2016?"
aggregated_answer, all_responses, context, sources = get_aggregated_answer_and_sources(question)

# Format the output in a readable way
formatted_context = "\n\n".join([f"File: {doc.metadata['file']}, Page {doc.metadata.get('page', 'Unknown')}: {doc.page_content}" for doc in context])

output = f"""
Final Aggregated Answer: {aggregated_answer}

Intermediate Answers:
Chroma: {all_responses.get('chroma', 'No valid answer found')}
FAISS: {all_responses.get('faiss', 'No valid answer found')}
Pinecone: {all_responses.get('pinecone', 'No valid answer found')}

Context used for answer:
{formatted_context}
"""

print(output)


Parent run 5a05e631-2f81-4ee4-a312-f450ec00314b not found for run c2a06fe2-7630-4de3-bbb5-3d525e5d76b0. Treating as a root run.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Parent run ff2c81d8-97f5-4a90-aa03-931379ba8f55 not found for run a9ca075e-a7aa-4022-b48e-5583f00f232b. Treating as a root run.


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Final Aggregated Answer: According to the retrieved context, the forecasts for U.S. crude oil production in the fourth quarter of 2016 were as follows:

- The current forecast for Q4 2016 was 8.94 million barrels per day.
- The previous forecast for Q4 2016 was 8.87 million barrels per day.

The percent change between the previous forecast and the current forecast for U.S. crude oil production in Q4 2016 was an increase of 0.8%.. The forecasts for U.S

Intermediate Answers:
Chroma: According to the retrieved context, the forecasts for U.S. crude oil production in the fourth quarter of 2016 were as follows:

- The current forecast for Q4 2016 was 8.94 million barrels per day.
- The previous forecast for Q4 2016 was 8.87 million barrels per day.

The percent change between the previous forecast and the current forecast for U.S. crude oil production in Q4 2016 was an increase of 0.8%.
FAISS: The forecasts for U.S. crude oil production in the fourth quarter of 2016 were:

- The current fo

In [5]:
# Use the get_aggregated_answer_and_sources function with the specified question
question = "How did the number of Chapter 11 filings in Connecticut change between 1994 and 2004?"
aggregated_answer, all_responses, context, sources = get_aggregated_answer_and_sources(question)

# Format the output in a readable way
formatted_context = "\n\n".join([f"File: {doc.metadata['file']}, Page {doc.metadata.get('page', 'Unknown')}: {doc.page_content}" for doc in context])

output = f"""
Final Aggregated Answer: {aggregated_answer}

Intermediate Answers:
Chroma: {all_responses.get('chroma', 'No valid answer found')}
FAISS: {all_responses.get('faiss', 'No valid answer found')}
Pinecone: {all_responses.get('pinecone', 'No valid answer found')}

Context used for answer:
{formatted_context}
"""

print(output)


Parent run 9fa9dfc3-216d-4e36-9888-763c3414ff9c not found for run 01c62925-716c-4149-9807-218b666d4bee. Treating as a root run.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Parent run e2e10478-856d-423b-a15b-66b247c7b72e not found for run b171338a-b548-413b-973f-f6869aea5ec7. Treating as a root run.


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Final Aggregated Answer: Between 1994 and 2004, the number of Chapter 11 filings in Connecticut decreased. In 1994, there were 216 filings, and by 2004, the number had dropped to 84 filings.. This indicates a significant reduction in the number of Chapter 11 filings over that ten-year period.

Intermediate Answers:
Chroma: Between 1994 and 2004, the number of Chapter 11 filings in Connecticut decreased. In 1994, there were 216 filings, and by 2004, the number had dropped to 84 filings.
FAISS: Between 1994 and 2004, the number of Chapter 11 filings in Connecticut declined. In 1994, there were 216 filings, and by 2004, the filings had decreased to 84. This indicates a significant reduction in the number of Chapter 11 filings over that ten-year period.
Pinecone: Between 1994 and 2004, the number of Chapter 11 filings in Connecticut decreased. In 1994, there were 216 filings, and by 2004, the number had dropped to 84 filings, indicating a significant decline over the ten-year period.

Con

In [6]:
# Use the get_aggregated_answer_and_sources function with the specified question
question = "What were the immediate actions taken by Nancy Powell and other volunteers following the Cosco Busan oil spill in San Francisco Bay?"
aggregated_answer, all_responses, context, sources = get_aggregated_answer_and_sources(question)

# Format the output in a readable way
formatted_context = "\n\n".join([f"File: {doc.metadata['file']}, Page {doc.metadata.get('page', 'Unknown')}: {doc.page_content}" for doc in context])

output = f"""
Final Aggregated Answer: {aggregated_answer}

Intermediate Answers:
Chroma: {all_responses.get('chroma', 'No valid answer found')}
FAISS: {all_responses.get('faiss', 'No valid answer found')}
Pinecone: {all_responses.get('pinecone', 'No valid answer found')}

Context used for answer:
{formatted_context}
"""

print(output)

Parent run f8d6e2eb-cee0-4f92-b6b3-69934e290c3a not found for run 734c1ed7-9b16-48f3-8cdf-ad2e94634e58. Treating as a root run.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Final Aggregated Answer: Following the Cosco Busan oil spill in San Francisco Bay, Nancy Powell and other volunteers took immediate action by:

1. Nancy Powell walked along the Bay from Emeryville to Berkeley to assess the situation, finding six oiled birds in Berkeley across from Shorebird Park, which she then protected.
2. Powell and other concerned individuals, including a kayaker, collected oiled birds. Despite lacking proper equipment, they borrowed nets from the Shorebird Nature Center and rescued a couple of birds on the beach.
3. The volunteers purchased over $500 worth of fishing nets and towels from local stores and spent the day helping rescue birds in Berkeley.
4. They organized a bird-rescue network, handling tasks such as closing off the shoreline, taking reports of oiled birds, and catching and transporting birds.
5. Over a week, Powell's group rescued over 50 birds and transported birds caught by others, including homeless individuals, for cleanup and care.. Walking al